# Assistant Demo

Forsøk på en rask test og demo av hvordan Assistants fungerer i OpenAI. Ideen er at man kan bruke dette til å lage en vector store som er søkbar. Kanksje ikke så nyttig for å spisse en CV men kan være interessant å se på. Spesielt siden disse vil være tilgjengelig i GUI når de er lagd, og visa versa. Men siden ChatGPT Teams lisensen er annerledes en openAI plattform der API vi bruker ligger så vil ikke disse være koblet.

### Setup

Her importerer vi bilbioteker, API nøkkler, og definerer en rolle beksrivelse som blir brukt senere (sånn at koden etterpå blir mer leselig)

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv("../.env")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
client = OpenAI()

In [3]:
Rolle_beskrivelse = """1   Navn på oppdrag /
prosjekt
LLM Engineer (mulig teamleder) til nytt KI Team
2   Kort beskrivelse av oppdraget
Sikt etablerer et nytt team som jobber med kunstig intelligens på tvers av virksomheten. Dette
vil støtte produkt-teamene i Sikt med kompetanse og gjennomføringskraft og realisere piloter
på ny og bedre funksjonalitet.
Vi ønsker med dette teamet og etablerere et sterkt fagmiljø for data science og KI i Sikt som
gjør oss i stand til å identifisere, planlegge og utnytte flere av mulighetene som KI åpner for i
produktene våre.
Selv om hovedoppgaven til teamet blir å realisere faktiske produkter og ny funksjonalitet, blir
det også en svært viktig å jobbe sammen med produkt-teamene for å identifisere de riktige
initiativene, samt å dokumentere og formidle både internt og eksternt om både læring, feil og
suksesser.
Vi har identifisert to ulike roller som blir viktig for oss fra starten, og hvor vi søker en erfaren
konsulent. En av disse rollene vil også ha funksjon som teamleder. Til stillingen som «LLM
engineer» («Large Language Model engineer») søker vi en erfaren konsulent med følgende
kvalifikasjoner:
Kvalifikasjoner
(dersom aktuell som teamleder) Erfaring med å lede og inspirere høytytende team
for å fremme et positivt arbeidsmiljø og høy leveransekraft.
\n
Evnen til å veilede og utvikle team-medlemmer, både faglig og personlig, for å styrke
teamets kompetanse og karriereutvikling.
\n
Kommunikasjon: Sterke muntlige og skriftlige kommunikasjonsevner for å forklare
komplekse analyser og konklusjoner til et ikke-teknisk publikum.
\n
Dyptgående programmeringskunnskap, fortrinnsvis Python.
\n
Praktisk erfaring og utforsking av innovativ bruk av LLM via API det siste året.
\n
Kunnskap og erfaring med open source språkmodeller, og fine-tuning av disse.
\n
Erfaring med RAG (Retrieval Augmented Generation) med LangChain eller
tilsvarende.
\n
Grunnleggende forståelse for maskinlæring og NLP (Natural Language Processing)
\n
Erfaring med å utvikle og vedlikeholde API-er.
\n
God forståelse og erfaring med git, CI/CD, skyteknologi og Docker eller kubernetes.
\n
Teamarbeid: Evnen til å jobbe effektivt i tverrfaglige team og samarbeide med andre
fagområder.
\n
Nysgjerrighet og lærevillighet: En evne til å holde seg oppdatert med de siste
trendene og teknologiene innen AI og maskinlæring.
\n
God skriftlig og muntlig fremstillingsevne på både norsk og engelsk
Arbeidsoppgaver:
\n
Sammen med produktteamene idémyldre, identifisere og planlegge nye piloter og
initiativer.
\n
Sammen med teamet utvikle ny funksjonalitet og tjenester med LLM og generativ AI.
\n
Formidling og historiefortelling om arbeidet, prosessen og resultatene underveis
\n
Bidra til godt samarbeid i produktteamet
Personlige egenskaper:
\n
Gode analytiske ferdigheter
\n
Høy gjennomføringsevne
\n
Presis i kommunikasjon
\n
Svært gode samarbeidsevner"""

## Definer en Assistant

Første steg er å lage en assistent. Vi må gi den et navn, en instruks, en model, og definere om det er noen verktøy den skal bruke. Her bruker vi file_search som verktøy, for å kunne søke i opplasstede filer.

In [31]:
cv_assistant = client.beta.assistants.create(
  name="CV Spisser Assistent",
  instructions="""
    Du er en selger av konsulenter. Bruk informasjonen hentet fra kunnskapsbasen av CV'er til å forslå en spisset CV intro tekst for en gitt prosjektbeskrivelse./n
    Du skal kun bruke info fra en gitt persons CV. Personen blir identifisert i prompten, sammen med prosjektbeskrivelsen. Den spisset CV intro teksten skal maks være 150 ord.
  """,
  model="gpt-4-turbo",
  tools=[{"type": "file_search"}]
)


Vi må også lage en vector store som assistenten skal bruke for å hente ut informasjon fra opplastede filer.

In [6]:

vector_store_CV = client.beta.vector_stores.create(
    name = "CV_store",
    expires_after={"anchor": "last_active_at", "days":3}
)


Her laster vi in alle filer som ligger i data mappen (når denne demoen ble lagd var det kun en fil der) og laster de opp til vector store. I denne prosessen blir filene delt opp i mindre biter for å bli konvertert til en vector. Dette er en begrensning pga embedding modelen som blir brukt for konverteringen. Denne modellen tar input av en vis størrelse. Dermed vil søk i den slik database hente de segmentene av filen opplastet som best passer med det man spør etter. Dermed blir ikke nødvendigvis hele CV'en brukt når den svarer.

In [21]:

file_streams = [open("../data/"+path, "rb") for path in os.listdir("../data/")]
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store_CV.id,
    files = file_streams
)

# Oppdater assistent med en kobling til vector store
cv_assistant = client.beta.assistants.update(
    assistant_id=cv_assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store_CV.id]}}
)


Man kommunisere til en assistent gjennom threads. Hver thread er ekvivalent med en chat i ChatGPT, og kan ha flere messages. En thread vil også da håndtere chat historikk.

Det finnes per i dag ikke en metode for å liste opp alle aktive threads, så hvis man skal gå tilbake til en thread så må dens id lagres selv.

In [23]:

thread_1 = client.beta.threads.create(messages=[])


Her lager vi en message som er koblet til en bestemt thread.

In [24]:

message_data = {
    "role": "user",
    "content": f"Bruk CV'en til Ask. Prosjektbeskrivelse: {Rolle_beskrivelse}"
}

client.beta.threads.messages.create(
    thread_id=thread_1.id,
    **message_data
)


Message(id='msg_x7TA7vcTUUVA148NsmKOMerM', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Bruk CV'en til Ask. Prosjektbeskrivelse: 1   Navn på oppdrag /\nprosjekt\nLLM Engineer (mulig teamleder) til nytt KI Team\n2   Kort beskrivelse av oppdraget\nSikt etablerer et nytt team som jobber med kunstig intelligens på tvers av virksomheten. Dette\nvil støtte produkt-teamene i Sikt med kompetanse og gjennomføringskraft og realisere piloter\npå ny og bedre funksjonalitet.\nVi ønsker med dette teamet og etablerere et sterkt fagmiljø for data science og KI i Sikt som\ngjør oss i stand til å identifisere, planlegge og utnytte flere av mulighetene som KI åpner for i\nproduktene våre.\nSelv om hovedoppgaven til teamet blir å realisere faktiske produkter og ny funksjonalitet, blir\ndet også en svært viktig å jobbe sammen med produkt-teamene for å identifisere de riktige\ninitiativene, samt å dokumentere og formidle både internt og eks

For å kjøre en thread å få en respons på en melding sent inn, må vi gjøre en run (man kan også sette opp en stream). Så kan vi liste opp alle messages gjor i en thread og parse responsen. Her er det lagt til litt støttekode for å formatere responsen til å legge ved siteringer.

In [25]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread_1.id, assistant_id=cv_assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread_1.id, run_id=run.id))

message_content = messages[0].content[0].text

# Hvis det finnes citations i output så blir de listet.
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

Ask Juhl Markestad er en dyktig og erfaren data scientist og data engineer med bred kunnskap og erfaring innen utvikling og implementering av avanserte AI-løsninger og datavitenskapelige prosjekter. Med en mastergrad i teoretisk fysikk fra Universitetet i Oslo, har han utviklet robuste ferdigheter i matematisk modellering og statistisk analyse. Ask har arbeidet med prosjekter som spenner over datavitenskap, maskinlæring, og kunstig intelligens i bransjer som finans, offentlig, og media. Han er sertifisert innen flere teknologier inkludert Azure Data Scientist og har praktisk erfaring med Python, AI-modellering med ChatGPT og RAG, samt teknologier som AWS og Azure  . Hans evne til teknologisk innovation og prosjektledelse kvalifiserer ham ypperlig for rollen som LLM Engineer hvor han vil kunne lede og inspirere teamet i utviklingen av ny funksjonalitet og tjenester ved bruk av de nyeste innen kunstig intelligens og maskinlæring.



# Clean up
Her er en kodesnutt for å fjerne en bestemt assistent og en bestemt thread.

In [ ]:
client.beta.assistants.delete(assistant_id=cv_assistant.id)
client.beta.threads.delete(thread_id=thread_1.id)

# OBS: Dette sletter alle assistenter koblet til dette API'et, inkludert de andre som bruker API'et har laget.
# Venligst ikke slett andre sitt arbeid. 

In [29]:
for assistant in client.beta.assistants.list():
    print(assistant.id)
    if assistant.tool_resources.file_search.vector_store_ids:
        client.beta.vector_stores.delete(vector_store_id=assistant.tool_resources.file_search.vector_store_ids)
    client.beta.assistants.delete(assistant_id=assistant.id)
client.beta.assistants.list()

### Slett alle active vector stores

In [30]:
for vector_store in client.beta.vector_stores.list():
    client.beta.vector_stores.delete(vector_store_id=vector_store.id)
client.beta.vector_stores.list()

### Slett alle opplastede file assosiert med en assistent

In [31]:
for file in client.files.list(purpose="assistants"):
    client.files.delete(file.id)
client.files.list(purpose="assistants")

SyncPage[FileObject](data=[], object='list', has_more=False)